In [1]:
import numpy as np
import pandas as pd

#simul parameters
nm = 3
nq = 1
T = 2

# Aux functions

Created this file because I forgot to commit the last file I\'ve used

Target: implement the MARSS Framework on Banbura Modugno 2010.


# MARSS

$$ x_t = B_t x_{t-1} + u_t + C_t c_t + w_t, \qquad where \quad w_t \sim MVN(0,Q_t) \qquad (1)$$
$$ y_t = Z_t x_t + a_t + D_t d_t + v_t, \qquad where v_t \sim MVN(0,R_t) \qquad (2)$$

* $x$ is a $m \times T$ matrix
* $w$ is $m \times T$ 
* $y$ is $n \times T$ 

Thus:

* m is the dimension of the state vector
* T is the sample length
* n is the total number of variables

# Banbura Modugno 2010

The first equation from Banbura Modugno corresponds to (2) and the second equation corresponds to (1)

![Banbura Modugno 2010](BanburaModugno.png)

# Dimensions

Lets build an example data set to work with

In [2]:
cols = ("m" + pd.DataFrame(list(range(0,nm))).astype(str))[0].tolist()
monthlyData = pd.DataFrame(np.random.rand(T,nm),columns=cols)
monthlyData

,m0,m1,m2
0,0.271045,0.440297,0.295701
1,0.305844,0.415225,0.331784


In [3]:
cols = ("q" + pd.DataFrame(list(range(0,nq))).astype(str))[0].tolist()
quarterlyData = pd.DataFrame(np.random.rand(T,nq),columns=cols)
quarterlyData

,q0
0,0.165650
1,0.831038


Mapping our dimensions:

In [4]:
T = monthlyData.shape[0]
T

2

In [5]:
n = (monthlyData.columns | quarterlyData.columns).shape[0]
n

4

In [6]:
nm = (monthlyData.columns).shape[0]
nm

3

In [7]:
nq = (quarterlyData.columns).shape[0]
nq

1

We will start with only one unobserved factor. Thus $f_t$ is $1 \times 1$.

In order to construct the following vector:

![State Vector](vec.png)

We need to remember that $\varepsilon_t^M$ is $n_M \times 1$ and $\varepsilon_t^Q$ is $n_Q \times 1$.

Also we have to include all 4 $\varepsilon_t^Q$'s lags

In [8]:
m = 5 + nm + 5 * nq
m

13

# Data Matrices

$y$ which is $n \times T$ will be the vertical stack of the monthly and quarterly variables

In [9]:
quarterlyData

,q0
0,0.165650
1,0.831038


In [10]:
monthlyData

,m0,m1,m2
0,0.271045,0.440297,0.295701
1,0.305844,0.415225,0.331784


In [11]:
y = pd.concat([quarterlyData.transpose(), monthlyData.transpose()], axis=0)
y

,0,1
q0,0.165650,0.831038
m0,0.271045,0.305844
m1,0.440297,0.415225
m2,0.295701,0.331784


Verification:

In [12]:
(y.shape[0] == n) and (y.shape[1] == T)

True

# Coefficients

## Z Matrix

![Z Matrix](zMatrix.png)

The $Z$ matrix is a $n \times m$

The matrix we see on Banbura and Modugno is clearly n x m

Firt we have to generate a vector of m different coefficients named by strings. Which will be Banbura's $\Lambda_M$

In [13]:
coefs = []
for el in monthlyData.columns:
    coefs.append(el + "_loading")
lambdaM = pd.DataFrame(coefs)

To complete the first line of their matrix:

In [19]:
line1 = lambdaM
line1 = pd.concat([line1, pd.DataFrame(np.zeros((nm,4)))], axis=1)

line1 = pd.concat([line1, pd.DataFrame(np.identity(nm))], axis=1)
missingDimension = m - line1.shape[1]
line1 = pd.concat([line1, pd.DataFrame(np.zeros((nm,missingDimension)))], axis=1)

line1

,0,0,1,2,3,0,1,2,0,1,2,3,4
0,m0_loading,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,m1_loading,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,m2_loading,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


we are not yet sure how to implement linear restrictions on parameters. Fow now we are assuming that we should use a string like "2*a"

We will first create lambdaQ

For now this code suffices. But if we choose to use more quarterly series or more unobserved factors we would have to change it.

In [24]:
coefs = []
for el in quarterlyData.columns:
    coefs.append(el + "_loading")
lambdaQ = pd.DataFrame(coefs)

line2 = lambdaQ
line2 = pd.concat([line2, "2*" + lambdaQ], axis=1)
line2 = pd.concat([line2, "3*" + lambdaQ], axis=1)
line2 = pd.concat([line2, "2*" + lambdaQ], axis=1)
line2 = pd.concat([line2, lambdaQ], axis=1)

line2 = pd.concat([line2, pd.DataFrame(np.zeros((nq,nm)))], axis=1)

line2 = pd.concat([line2, pd.DataFrame(np.ones((nq,1)))], axis=1)
line2 = pd.concat([line2, 2*pd.DataFrame(np.ones((nq,1)))], axis=1)
line2 = pd.concat([line2, 3*pd.DataFrame(np.ones((nq,1)))], axis=1)
line2 = pd.concat([line2, 2*pd.DataFrame(np.ones((nq,1)))], axis=1)
line2 = pd.concat([line2, pd.DataFrame(np.ones((nq,1)))], axis=1)

line2

,0,0,0,0,0,0,1,2,0,0,0,0,0
0,q0_loading,2*q0_loading,3*q0_loading,2*q0_loading,q0_loading,0.0,0.0,0.0,1.0,2.0,3.0,2.0,1.0


In [25]:
line1.columns = list(range(0,line1.columns.shape[0]))
line2.columns = list(range(0,line2.columns.shape[0]))

Z = pd.concat([line1,line2], axis=0, ignore_index=True)
Z

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,m0_loading,0,0,0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,m1_loading,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,m2_loading,0,0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,q0_loading,2*q0_loading,3*q0_loading,2*q0_loading,q0_loading,0.0,0.0,0.0,1.0,2.0,3.0,2.0,1.0


## B Matrix

![B matrix](bMatrix.png)

$B$ matrix is $m \times m$

And m is eleven

Since we're using only one unobserved factor, for the time being $A_t$ is a scalar and $r=1$.

In [72]:
line1 = pd.DataFrame(["A1"])
line1 = pd.concat([line1, pd.DataFrame(np.identity(4))], axis=0)
# line1 = pd.concat([line1, pd.DataFrame(np.zeros((5,1)))], axis=1, ignore_index=True)
line1[line1.columns.shape[0]]=0


alphaM = "alphaM_" + pd.DataFrame([range(0,nm)]).astype(str) 
alphaM = alphaM.transpose()[0]
alphaM = pd.DataFrame(np.diag(alphaM), )

alphaCol = pd.concat([pd.DataFrame(np.zeros((line1.shape[0],alphaM.shape[1]))), alphaM])

line1 = pd.concat([line1, pd.DataFrame(np.zeros(alphaM.shape))], axis=0, ignore_index=True)

line1 = pd.concat([line1, alphaCol], axis=1, ignore_index=True)

line1

ValueError: Shape of passed values is (8, 11), indices imply (8, 8)

In [73]:
alphaCol

(8, 3)

In [74]:
line1.s

(8, 5)